# litigation에서 firm-lv 변수 뽑기

### 1. litigation에 gvkey, year 붙이기
- litigation 정리
- gvkey 부착

### 2. 변수 제작
- +3, +4, +5 years
- counting (공, 수, 전체)

### 1. litigation - gvkey year
- 2006 ~ 2015 case 추출 (patent_case)
- patent_case, names join
- party type 정리 (groupby party_type count() 해서 버릴 거 버림)

__1. 추출 & join__

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

os.chdir('E:/apps/db_table')

In [2]:
df = pd.read_csv('patent_cases.csv')
list(df)

['case_row_id',
 'case_number',
 'pacer_id',
 'case_name',
 'court_name',
 'assigned_to',
 'referred_to',
 'case_cause',
 'jurisdictional_basis',
 'demand',
 'jury_demand',
 'lead_case',
 'related_case',
 'settlement',
 'date_filed',
 'date_closed',
 'date_last_filed']

In [3]:
df = df.loc[:,['case_row_id','date_filed']]
df.dropna(how='any',inplace=True)
df['year'] = df['date_filed'].apply(lambda x: int(x[:4]),1)
df = df.loc[df['year']>2005,:]
len(df)

5654

In [4]:
df1 = pd.read_csv('names.csv')
list(df1)

['case_row_id',
 'case_number',
 'party_row_count',
 'party_type',
 'name_row_count',
 'name']

In [5]:
df1 = df1.loc[:,['case_row_id','party_type','name']]
df = df.merge(df1,how='left',on='case_row_id')
len(df)

66303

In [6]:
df.to_csv('lit_temp.csv',index=False)

__2. party type 정리__

In [2]:
df = pd.read_csv('lit_temp.csv')

In [3]:
ptype = df.loc[:,['party_type','count']]
ptype['count']=0
ptype = ptype.groupby('party_type', as_index=False).count()
ptype.sort_values(by='count',ascending=False,inplace=True)

In [4]:
sum(ptype['count'])

66049

In [5]:
sum(ptype['count'][0:40])

65986

- 공격: plaintiff, count claim (consol, 3rd party, ...)
- 수비: defendant, counter defendant (consol, 3rd party, ...)
  
__=> 전체 통합 버전, counter 분리 버전 만들기__ 

__전체 통합 (공, 수)__

In [6]:
attack = ['Counter Claimant','Plaintiff','Counter-claimant','Consol Counter Claimant','Consol Plaintiff','ThirdParty Plaintiff',
          'Counter Plaintiff','Counter-Claimant','Third Party Plaintiff','3rd party plaintiff','Counterclaim Plaintiff']
defend = ['Defendant','Counter Defendant','Consol Defendant','Counter-defendant','ThirdParty Defendant','Consol Counter Defendant',
         'Third Party Defendant','3rd party defendant','Counter-Defendant','Consolidated Defendant','Counterclaim Defendant']

atk = df.loc[df['party_type'].isin(attack),:]
dfd = df.loc[df['party_type'].isin(defend),:]

len(atk), len(dfd)

(28494, 35720)

In [7]:
list(atk)

['case_row_id', 'date_filed', 'year', 'party_type', 'name']

In [8]:
del atk['case_row_id']
del dfd['case_row_id']

In [9]:
bio = pd.read_csv('litigation_gvkey_BIO.csv')
ee = pd.read_csv('litigation_gvkey_EE&IT.csv')

bioa = bio.merge(atk,how='left',on='name')
biod = bio.merge(dfd,how='left',on='name')

eea = ee.merge(atk,how='left',on='name')
eed = ee.merge(dfd,how='left',on='name')

In [10]:
l = [bioa,biod,eea,eed]

for df in l:
    del df['name']
    df.dropna(how='any',inplace=True)
    df.drop_duplicates(inplace=True)

In [11]:
print len(bio), len(bioa), len(biod)

2766 395 431


In [12]:
print len(ee), len(eea), len(eed)

5137 1973 2548


__counter 버전__

In [15]:
acnt = ['Counter Claimant','Counter-claimant','Consol Counter Claimant','Counter Plaintiff','Counter-Claimant','Counterclaim Plaintiff']
dcnt = ['Counter Defendant','Counter-defendant','Consol Counter Defendant','Counter-Defendant','Counterclaim Defendant']

In [28]:
def counter_a(party):
    dum=1 if party in acnt else 0
    return dum

def counter_d(party):
    dum=1 if party in dcnt else 0
    return dum

a = [bioa,eea]
d = [biod,eed]

for df in a:
    df['counter'] = df['party_type'].apply(counter_a,1)
        
for df in d:
    df['counter'] = df['party_type'].apply(counter_d,1)

In [29]:
bioa.to_csv('E:/apps/bioa.csv',index=False)
biod.to_csv('E:/apps/biod.csv',index=False)
eea.to_csv('E:/apps/eea.csv',index=False)
eed.to_csv('E:/apps/eed.csv',index=False)

### 2. 변수 제작
- +3, +4, +5 years
- groupby gvkey, year
- 3, 4, 5년씩 묶기

In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math
os.chdir('E:/apps')

In [2]:
bioa = pd.read_csv('bioa.csv')
biod = pd.read_csv('biod.csv')
eea = pd.read_csv('eea.csv')
eed = pd.read_csv('eed.csv')

list(bioa)

['gvkey', 'date_filed', 'year', 'party_type', 'counter']

In [3]:
def get_lit_var(df,types,interval):
    df = df.loc[:,['gvkey','year']]
    df['count'] = 0
    df = df.groupby(['gvkey','year'], as_index=False).count()
    
    f_df = pd.DataFrame(columns=['gvkey', 'year'])
    gvkey = df['gvkey'].unique()
    
    for key in gvkey:
        for y in range(2006,2016):
            f_df = f_df.append({'gvkey': key,'year':y}, ignore_index=True)
            
    f_df = f_df.merge(df,how='left',on=['gvkey','year'])
    f_df.fillna(0,inplace=True)
    
    df = pd.DataFrame(columns=['gvkey', 'year','lit_'+str(types)+'_'+str(interval)])

    for key in gvkey:
        tmp = f_df.loc[f_df['gvkey']==key,:]

        for i in range(0,len(tmp)-interval+1):
            lit = sum(tmp['count'][i:i+interval])        
            df = df.append({'gvkey': key,'year':tmp.iloc[i,1]-1,'lit_'+str(types)+'_'+str(interval):lit}, ignore_index=True)
            
    df = df.apply(lambda x: map(int,x),axis=1,reduce=False)
    
    return df

In [4]:
for i in range(3,6):
    get_lit_var(bioa,types='a',interval=i).to_csv('feature/lit_a_'+str(i)+'_bio.csv',index=False)
    get_lit_var(eea,types='a',interval=i).to_csv('feature/lit_a_'+str(i)+'_EE&IT.csv',index=False)
    get_lit_var(biod,types='d',interval=i).to_csv('feature/lit_d_'+str(i)+'_bio.csv',index=False)
    get_lit_var(eed,types='d',interval=i).to_csv('feature/lit_d_'+str(i)+'_EE&IT.csv',index=False)